In [1]:
#@ Importing the required libraries
import pandas as pd
import numpy as np

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
#@ Loading the datasets
data = pd.read_csv('./dataset/covid_toys.csv')
data.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [4]:
#@ train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns = ['has_covid']), data['has_covid'],
                                                   test_size = 0.2)

In [5]:
X_train

,age,gender,fever,cough,city
26,19,Female,100.0,Mild,Kolkata
32,34,Female,101.0,Strong,Delhi
52,47,Female,100.0,Strong,Bangalore
46,19,Female,101.0,Mild,Mumbai
5,84,Female,NaN,Mild,Bangalore
...,...,...,...,...,...
72,83,Female,101.0,Mild,Kolkata
78,11,Male,100.0,Mild,Bangalore
57,49,Female,99.0,Strong,Bangalore
94,79,Male,NaN,Strong,Kolkata


### General way

In [6]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [7]:
#@ OrdinalEncoding for 'cough' column
oe = OrdinalEncoder(categories = [['Mild', 'Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

#also for the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [8]:
#@ OneHotEncoding for ---> 'gender' & 'city'
ohe = OneHotEncoder(drop = 'first', sparse = False)     # for multicollinearity we drop one column 
X_train_gender_city = ohe.fit_transform(X_train[['gender', 'city']])

# also for the tests data
X_test_gender_city = ohe.fit_transform(X_test[['gender', 'city']])

X_train_gender_city.shape

(80, 4)

In [9]:
# Extracting the age
X_train_age = X_train.drop(columns = ['gender', 'fever', 'cough', 'city']).values

# also for the test data
X_test_age = X_train.drop(columns = ['gender', 'fever', 'cough', 'city']).values

X_train_age.shape

(80, 1)

In [10]:
X_train_transformed = np.concatenate((X_train_age, X_train_fever, X_train_gender_city, X_train_cough),axis = 1)
# also for the test data
# X_test_transformed = np.concatenate((X_test_age, X_test_fever, X_test_gender_city, X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

In [12]:
# also for the test data
# X_test_transformed = np.concatenate((X_test_age, X_test_fever, X_test_gender_city, X_test_cough),axis = 1)
# X_test_transformed.shape

### Using ColumnTransformer

In [14]:
from sklearn.compose import ColumnTransformer

In [16]:
transformer = ColumnTransformer(transformers = [
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories = [['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(sparse = False, drop = 'first'), ['gender', 'city'])
], remainder = 'passthrough')

In [18]:
transformer.fit_transform(X_train).shape

(80, 7)